# VPP Optimalizáció Modell 1.0

* Jelölések
    * $i,j,k$: VPP résztvevők indexe (ha több lehet belőlük)
    * $t$: a diszkrét időintervallum indexe (az időt óránként/félóránként/negyedóránként tekintjük egy 24 órás időtartamra)

---

* Változók
    * $g_{i,t}$: gázmotorok 
    * $s_{i,t}, c_{i,t}, d_{i,t}$: energiatárolók (tároló, töltés, fogyasztás)
    
---

* Konstansok (egyelőre generáljuk őket)
    * $\overline{r}_{i,t}$: megújuló energiaforrások várható maximális teljesítménye
    * $l_{t}$: fogyasztó
    * $d_{t}$: energiaigény (ennyit energiát szeretnénk előállítani a $t$ időintervallumban a VPP-vel)
---

* Változók száma
    * $N_g$: gázmotorok száma
    * $N_s$: energiatárolók száma
    * $N_r$: megújuló energiaforrások száma
    * $T$: diszkrét időintervallumok száma.
    
    Minden $t$ diszkrét időintervallumban, minden $i$ entitás index mellett a szabályozható VPP résztvevők teljesítményéhez tartozik egy változó. A változókra (lineáris) kényszereket írunk fel, a költségfüggvény (lineáris) függvénye a változóknak (az idődiszkretizáció mellett). Feltesszük, hogy a diszkrét időpillanatok ekvidisztánsak, azaz az időintervallumok azonos hosszúságúak, emiatt feltételezhetjük, hogy $t = 1,2,\dots,T$. 
    A modell változóinak a száma:
    $$
        T \cdot (N_g + N_r + 3 N_s)
    $$

---

### Gázmotorok

* Paraméterek
    * $g_{max}$: a gázmotor maximális teljesítménye
    * $\Delta g_{max}^+$: az egy időintervallumban megvalósítható maximális teljesítmény növelés mértéke
    * $\Delta g_{max}^-$: az egy időintervallumban megvalósítható maximális teljesítmény csökkentés mértéke
    * $\gamma$: a gázmotor használatának fajlagos (lineáris) költsége

* Kényszerfeltételek    
    * Korlátosság
    $$
        0 \leq g_{i,t} \leq g_{max}
    $$
    * Korlátos szabályozhatóság
    $$
        -\Delta g_{max}^- \leq g_{i,t} - g_{i,t-1} \leq \Delta g_{max}^+
    $$
    * Minimális futási, leállítás idő
     
     *Ez MILP problémára vezet, egyelőre csak az LP problémával foglalkozunk, ezt a kényszert később vesszük a modellbe.*
    
* Költség
    * A gázmotorok összköltsége
    $$
        C_g = \sum_i \sum_t \gamma g_{i,t}
    $$

---

### Energiatárolók
* Változók:
    * $s_{i,t}$: az energiatárolóban tárolt energia (**s**torage)
    * $c_{i,t}$: energia töltése a tárolóba (**c**harge)
    * $d_{i,t}$: energia felhasználása a tárolóból (**d**ischarge)
    
* Paraméterek
    * $s_{min}$: az energiatároló minimális töltöttsége ($s_{min}$ alá nem meríthetjük a tárolót)
    * $s_{max}$: az energiatároló maximális töltöttsége
    * $c_{max}$: maximális töltés (legfeljebb ennyi energiát tölthetünk időegységenként a tárolóba)
    * $d_{max}$: maximális energiahasználat (legfeljebb ennyi energiát használhatunk időegységenként a tárolóból)
    * $\varphi_c$: a töltés veszteségi faktora (az energia $\varphi_c$ hányada elvész töltéskor)
    * $\varphi_d$: az energiahasználat veszteségi faktora (az energia $\varphi_d$ hányada elvész a használat során)
    * $\sigma_c$: a töltés fajlagos költsége
    * $\sigma_d$: a használat fajlagos költsége

* Kényszerfeltételek
    * Korlátosság
    $$
        \begin{array}{l}
            s_{min} \leq s_{i,t} \leq s_{max} \\
            0 \leq c_{i,t} \leq c_{max} \\
            0 \leq d_{i,t} \leq d_{max} \\
        \end{array}
    $$    
    * Kontinuitás
    $$
        \begin{array}{l}
            s_{i,t} = s_{i,t-1} + (1-\varphi_c) c_{i,t} - (1+\varphi_d) d_{i,t}
        \end{array}
    $$    
    * Kizárólagosság (az energiatároló minden pillanatban vagy felvesz vagy lead energiát) 
    *Nemlineáris kényszer, egyelőre nem foglalkozunk vele, lehet, hogy később sem.*
    
    
* Költség
Az $i$-edik tároló költsége:
$$
    \sum_t (1-\varphi_c) c_{i,t} \sigma_c + (1+\varphi_d) d_{i,t} \sigma_d
$$
Az energiatárolók összköltsége:
$$
    C_s = \sum_i \sum_t (1-\varphi_c) c_{i,t} \sigma_c + (1+\varphi_d) d_{i,t} \sigma_d
$$    

---

### Megújuló energiaforrások

* Paraméterek
    * $\overline{r}_{i,t}$: az energiaforrás $t$ időintervallumban várható teljesítménye
    * $\Delta r_{max}^+$: az egy időintervallumban megvalósítható maximális teljesítmény növelés mértéke
    * $\Delta r_{max}^-$: az egy időintervallumban megvalósítható maximális teljesítmény csökkentés mértéke
    * $\rho$: az energiaforrás használatának fajlagos (lineáris) költsége
* Feltételek    
    * Korlátosság
    $$
        0 \leq r_{i,t} \leq \overline{r}_{i,t} \\
    $$
    * Korlátos szabályozhatóság
    $$
        -\Delta r_{max}^- \leq r_{i,t} - r_{i,t-1} \leq \Delta r_{max}^+
    $$
* Költség
    * A megújuló energiaforrások összköltsége
    $$
        C_r = \sum_i \sum_t \rho r_{i,t}
    $$

---

### Fogyasztó

A fogyasztó $l_t$ értékeit adottnak feltételezzük. Azt a konvenciót alkalmazzuk, hogy az $l_t > 0$ azt jelenti, hogy a gyár a $t$-edik időintervallumban $l_t$ mennyiségű energiát fogyaszt. 

---

###  Energiaigény és összenergia

Adottnak feltételezzük a minden $t$ időintervallumra vonatkozó energiaigényt. Minden pillanatban *pontosan* $d_t$ mennyiségű energiát szeretnénk biztosítani. 

* Kényszerfeltétel
$$
    \sum_i g_{i,t} + \sum_{j} d_{j,t} + \sum_k r_{k,t} - l_t = d_t
    
$$

---

### Összköltség

A feladatban a
$$
    C = C_g + C_s + C_r
$$
összköltséget minimalizáljuk a fent részletezett kényszerek fennállása mellett.